In [1]:
# run the cell first for all setup

import re
from re import search
import os, sys
from os import walk
import traceback
import subprocess
import pandas as pd
import time
import math
import numpy as np
import requests
from bs4 import BeautifulSoup
import json

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys

def PROTEINSEQ (ptcode):
    #ptcode='P08908'
    try:
        web='https://www.uniprot.org/uniprot/'+ptcode+'.fasta'
        pg1=requests.get(web)
        proseq_fa=''
        proseq_fa=proseq_fa.join(pg1.text.split('\n')[1:])
    except:
        print ('PROTEINSEQ ptcode ERROR: ',ptcode,' xxxxxxxx ')
        proseq_fa=''
    return (proseq_fa)

def PROTEINCODE (prot):
    web='https://rest.uniprot.org/uniprotkb/'+prot
    pg1 = requests.get(web)
    pg2 = json.loads(pg1.text)
    try:
        ptcode=pg2['primaryAccession']
    except:
        try:
            ptcode=pg2['secondaryAccessions']
            ptcode=re.sub('\[','',ptcode)
            ptcode=re.sub('\]','',ptcode)
            ptcode=re.sub('\'','',ptcode)
        except:
            ptcode=''
            print (' cannot get ptcode for ', prot)
    try:
        protitle=pg2['uniProtkbId']
    except:
        protitle=''

    return (protitle, ptcode)

def RDkitECFP(smiles):
    molecular=Chem.MolFromSmiles(smiles)
    fp = [ i for i in AllChem.GetMorganFingerprintAsBitVect(molecular,3,nBits=1024) ]
    return(fp)

def PROTEIN_ALIGN (workpath, aliinput):
    ### work_path & input FASTA seq -> return align file name + AA coding
    os.chdir(workpath)
    output_txt=re.sub(r'.txt','',aliinput)+'_align.out'
    output_csv=re.sub(r'.out','',output_txt)+'.csv'
    print ("Now is in :", os.getcwd(),'\n\n' )
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())))
    cmd='./bin/MUSCLE/muscle_v5.0.1428_linux -align '+ aliinput + ' -output ' + output_txt
    print (cmd)
    subprocess.run(cmd, shell=True) #, capture_output=True)

    if not os.path.isfile(output_txt):
        cmd='./bin/MUSCLE/muscle3.8.31_i86linux64 -in '+ aliinput + ' -out ' + output_txt
        print (cmd)
        subprocess.run(cmd, shell=True)

    aminoacid_table1={'X':'0','-':'0','C':'1','G':'1','P':'1','A':'2','V':'2','I':'2','L':'2','M':'2','F':'2','Y':'2','W':'2','S':'3','T':'3','N':'3','Q':'3','D':'4','E':'4','R':'5','H':'5','K':'5'}

    fileread = open(output_txt,'r')
    index_num=0
    proname=''
    proseq_fa1=pd.DataFrame()
    proseq_code1=list()
    proseq_code2=list()
    proseq_code3=list()
    proseq_code9=list()
    for i,lineread in enumerate (fileread):
        lineread = re.sub('\n','',lineread)
        if re.search('>',lineread) :
            lineread=re.sub('>','',lineread)
            lineread=re.split(',|\s+',lineread)
            if i<1:
                procode=lineread[1]
                proname=lineread[0]
            else:
                proseq_fa1[index_num]=[procode,proname,proseq_code1,proseq_code2,proseq_code3,proseq_code9]
                index_num=index_num+1
                procode=lineread[1]
                proname=lineread[0]
                proseq_code1=list()
                proseq_code9=list()
        else:
            for aaread in  lineread:
                proseq_code1.append(aminoacid_table1[aaread])
                proseq_code9.append(aaread)

    proseq_fa1[index_num]=[procode,proname,proseq_code1,proseq_code2,proseq_code3,proseq_code9]
    proseq_fa1.to_csv(output_csv,index=False, header=False)
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())),'\n\n')
    return (output_csv)

def PADELdiscriptor (workpath,toPADEL_file,smiles_label,chemical_label):
    os.chdir(workpath)
    timestamp=time.strftime("%m%d%H%M",time.localtime(time.time()))
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())))
    padelfilein=workpath+'/'+'padlein_molecule'+timestamp+'.smi'
    padelfileout=workpath+'/'+'padleout_molecule_1d2ddescriptor'+timestamp+'_'+toPADEL_file
    padel0=pd.read_csv(toPADEL_file, low_memory=False)
    padel2=pd.concat([padel0[smiles_label],padel0[chemical_label]], axis=1)
    padel2=padel2.dropna(axis=0,subset=[smiles_label,chemical_label])
    padel2.to_csv(padelfilein, sep='\t', index=False, header=False)
    cmd='xvfb-run --auto-servernum --server-num=5 java -jar /home1/drhuangwc/bin/PaDEL-Descriptor/PaDEL-Descriptor.jar'+' -dir '+padelfilein +' -file '+ padelfileout +' -maxruntime 3600000 -2d -threads -1 -removesalt -detectaromaticity -standardizenitro'  #
    print (cmd)
    subprocess.run(cmd, shell=True) #, capture_output=True)
    print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())))
    return (padelfileout)



def Agonist_Anagonist_LABEL(dfAgonistAnagonist,AgonistAnagonist_label):
    # Agonist and Anagonist [1,-1]
    dfAgonistAnagonist=dfAgonistAnagonist.sort_values(by=['Receptor'])
    dfAgonistAnagonist=dfAgonistAnagonist.reset_index(drop=True)
    for i, assay_description in enumerate (dfAgonistAnagonist['Assay Description']):
        if re.search ('ntagoni', assay_description):
            dfAgonistAnagonist.at[i,AgonistAnagonist_label]=-1
        elif re.search ('goni', assay_description):
            dfAgonistAnagonist.at[i,AgonistAnagonist_label]=1
        else:
            dfAgonistAnagonist.at[i,AgonistAnagonist_label]=9
    print ("Agonist: " , dfAgonistAnagonist.loc[dfAgonistAnagonist[AgonistAnagonist_label]==1].shape, "  Antagonist: " , dfAgonistAnagonist.loc[dfAgonistAnagonist[AgonistAnagonist_label]==-1].shape)
    return(dfAgonistAnagonist)


def reorder_columns(df_temp_reorder, column_name, position):
    df_temp_reorder_copy=df_temp_reorder.copy()
    try:
        insert_column = df_temp_reorder[column_name]
        df_temp_reorder=df_temp_reorder.drop(columns=[column_name], axis=1)
        df_temp_reorder.insert(loc=position, column=column_name, value=insert_column)
    except:
        df_temp_reorder=df_temp_reorder_copy
    return (df_temp_reorder)


def column_del(df_temp_del, column_drop_list,dfNAME='df'):
    for i, columndel in enumerate (column_drop_list):
        try:
            df_temp_del=df_temp_del.drop(columns=[columndel], axis=1)
        except:     # KeyError
            print (' ')
    return (df_temp_del)


def code_to_column(df_temp_cc,PROTEIN_code,PROTEIN_code_drop, proteinname,ec50_label,AgonistAnagonist_label):
    # ec50_label='ActivityValue_log_EC50'
    # AgonistAnagonist_label='AgonistP1AnagonistN1_label'   # Agonist and Anagonist [1,-1] 
    df_temp_cc=df_temp_cc.rename(columns={'ActivityValue':ec50_label}) 
    df_temp_cc=column_del(df_temp_del=df_temp_cc, column_drop_list=['index','Protein_id','Protein_code9',  'Protein_title', 'Protein_seq', 'Receptor', 'p-value(-log)', 'ActivityType', 'Assay Description',  'Smiles'],dfNAME=proteinname)
    df_temp_cc=column_del(df_temp_del=df_temp_cc, column_drop_list=['Molweight', 'RotBonds','H don','H acc','LogP'],dfNAME=proteinname)
    df_temp_cc=column_del(df_temp_del=df_temp_cc, column_drop_list=PROTEIN_code_drop,dfNAME=proteinname)
    df_temp_cc=reorder_columns(df_temp_reorder=df_temp_cc, column_name=PROTEIN_code, position=0)
    df_temp_cc=reorder_columns(df_temp_reorder=df_temp_cc, column_name='ECFP', position=1)
    i=0
    for i, ec50 in enumerate (df_temp_cc[ec50_label]):
        try:
            ec50=float(ec50)
            if ec50 == 0:
                df_temp_cc.at[i, ec50_label]=0
                df_temp_cc.at[i, AgonistAnagonist_label]=0
            else: 
                df_temp_cc.at[i, ec50_label]=math.log(ec50,10)
        except:
            df_temp_cc.at[i, ec50_label]=0
            df_temp_cc.at[i, AgonistAnagonist_label]=6
        
    code2columns=[PROTEIN_code, 'ECFP','padel1d2d' ]
    for code in code2columns:
        i=0
        print('\n',time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())))
        print ('       ',proteinname,' code_to_column for : ' ,code)
        df_temp_cc=df_temp_cc.reset_index(drop=True)
        for i, read_code_list in enumerate (df_temp_cc[code]):
            if pd.isna(read_code_list) or read_code_list=='':
                df_temp_cc.at[i,'DEL_DATA']='DEL_DATA'
                print ('skip ',i,code)
                continue
            else:
                try:
                    code_extand = [re.sub('\s|\[|\]|\'','',x) for x in read_code_list.split(',')]
                except:
                    print (' *** ',proteinname,' has no ',code,' -> skip ')
                    df_temp_cc.at[i,'DEL_DATA']='DEL_DATA'
                    continue
                if len(code_extand)<3:
                    df_temp_cc.at[i,'DEL_DATA']='DEL_DATA'
                    print (' *** ',proteinname,' has wrong coding ',code,' -> skip ')
                    continue
                j=0
                for j, coding in enumerate (code_extand):
                    new_columnname=code+'_'+str(j)
                    df_temp_cc.at[i, new_columnname]=coding 

    try:
        df_temp_cc=df_temp_cc.loc[df_temp_cc['DEL_DATA']!='DEL_DATA']
    except:
        df_temp_cc=df_temp_cc
            
    df_temp_cc=column_del(df_temp_del=df_temp_cc, column_drop_list=code2columns,dfNAME=proteinname)
    df_temp_cc=column_del(df_temp_del=df_temp_cc, column_drop_list=['DEL_DATA'],dfNAME=proteinname)
    df_temp_cc=reorder_columns(df_temp_reorder=df_temp_cc, column_name=AgonistAnagonist_label, position=len(df_temp_cc.columns)-1)
    df_temp_cc=reorder_columns(df_temp_reorder=df_temp_cc, column_name=ec50_label, position=len(df_temp_cc.columns)-1)
    df_temp_cc=df_temp_cc.reset_index(drop=True)
    return(df_temp_cc,ec50_label)


homepath=str(os.getcwd())





### CODE: Main



In [ ]:
print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())),'\n')

timestamp=time.strftime("%Y%m%d",time.localtime(time.time()))
starttime=time.time()
starttime1=time.time()

ec50_label='ActivityValue_log_EC50'  
AgonistAnagonist_label='AgonistP1AnagonistN1_label'   # Agonist and Anagonist [1,-1] 


runfolder='Human_AgonistAntagonist'
workpath=homepath+'/'+runfolder

os.chdir(homepath)
cmd='mkdir '+runfolder
os.system(cmd)

print("Load original database")
os.chdir(homepath+'/ori_db')
df1=pd.DataFrame()
for f_i in range (0,11):
    filename='gpcrdb_2021_'+str(f_i)+'.csv'
    dfcut=pd.read_csv(filename)
    df1=pd.concat([df1,dfcut],axis=0)
    df1=df1.reset_index(drop=True)

fileoutAgonistAntagonist='GPCR_AgonistAntagonist20220915.csv' #'GPCR_AgonistAntagonist'+timestamp+'.csv'
fileoutAgonistAntagonist_pro='GPCR_AgonistAntagonist_protein20220915.csv' #'GPCR_AgonistAntagonist_protein'+timestamp+'.csv'
fileprotseq='proteinseq20220915.txt' #'proteinseq'+timestamp+'.txt'
fileprotseqcode123_out='GPCR_AgonistAntagonist_protein_align_ptcode123_20220915.csv' #'GPCR_AgonistAntagonist_protein_align_ptcode123_'+timestamp+'.csv'
fileprotseqcode123_1d2dpadel_out='GPCR_AgonistAntagonist_protein_align_ptcode123_padel1d2d20220921_rdkECFP1019.csv'

####################
starttime1=time.time()

df2=column_del(df_temp_del=df1, column_drop_list=["IND","Select all","Unit","Purchasable","ActivityRelation","Assay Type",'p-value(-log)'],dfNAME='oridbfile')

df3=df2.loc[df2["Species"].str.contains('uman')]
df3=df3.reset_index(drop=True)
###
print ('Insert Human Agonist_Anagonist_LABEL ')

###  Human Agonist and Antagonist
df5=Agonist_Anagonist_LABEL(df3,AgonistAnagonist_label=AgonistAnagonist_label)    ### call Agonist_Anagonist_LABEL
df5.to_csv(fileoutAgonistAntagonist,index=False,encoding='utf-8') ##


print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n  duration = ', (time.time()-starttime1)/3600, ' hrs \n' )

df6=df5.loc[(df5[AgonistAnagonist_label]==1) | (df5[AgonistAnagonist_label]==-1) ]

print("Process Human Agonist and Antagonist ->  protein  : ",  len(df5), len (df6))

df6.insert(loc=0, column='Protein_id', value='')
df6.insert(loc=1, column='Protein_name', value='')
df6.insert(loc=2, column='Protein_title', value='')
df6.insert(loc=3, column='Protein_seq', value='')
df6['ECFP']=''

### Insert protCODE protSEQ protTITLE RDkitECFP fileprotseq='protseq.txt'
starttime1=time.time()
print("Insert protCODE protSEQ protTITLE SMILE ECFP in to the GPCRdb file ")
print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n')

file_w=open(fileprotseq,'a')
db_seq=''
protNAME0=''
p=0

df6=df6.sort_values(by=['Receptor'])
df6=df6.reset_index(drop=True)
for i, protNAME in enumerate (df6['Receptor']): 
    protNAME1=re.sub('\s+','', protNAME)
    protNAME2=re.sub('_human','',protNAME1)
    protNAME2=protNAME2.upper()
    if protNAME1 != protNAME0:
        protTITLE, protCODE = PROTEINCODE(protNAME1)      ### call PROTEINCODE
        protSEQ = PROTEINSEQ(protCODE)                    ### call PROTEINSEQ
        db_seq='>'+protNAME2+' '+protCODE+' '+str(p)+',\n'+protSEQ+'\n'
        file_w.write(db_seq)
        protNAME0 = protNAME1
        p=p+1

    df6.at[i,'Protein_id']=protCODE
    df6.at[i,'Protein_name']=protNAME2
    df6.at[i,'Protein_title']=protTITLE
    df6.at[i,'Protein_seq']=protSEQ
    
    ChEMBLID=df6.at[i,'ChEMBL ID']

    ecfpconv=''
    
    if ecfpconv=='':
        smiles=df6.at[i,'Smiles']
        smiles=re.sub('\s+','',smiles)
        try:
            ecfpconv=RDkitECFP(smiles)
        except:
            ecfpconv=''

                
    if ecfpconv=='':  
        df6.at[i,AgonistAnagonist_label]=9
                            
    df6.at[i,'ECFP']=ecfpconv
        
    if i%50 ==0:
        df6.to_csv(fileoutAgonistAntagonist_pro,index=False,encoding='utf-8') ##
        if i%500 ==0:
            print ('progress: ', i)

df6.to_csv(fileoutAgonistAntagonist_pro,index=False,encoding='utf-8') ##
print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n  duration = ', (time.time()-starttime)/3600, ' hrs \n' )

#############

file_w.close()

############# PROTEIN_ALIGN #############

starttime1=time.time()
print("call PROTEIN_ALIGN")
print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n')

os.chdir(workpath)

proteinalignout=PROTEIN_ALIGN (workpath,fileprotseq)                    ### call PROTEIN_ALIGN

print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n  duration = ', (time.time()-starttime)/3600, ' hrs \n' )

df7=pd.read_csv(proteinalignout, low_memory=False)

df6['Protein_code1']=''
df6['Protein_code9']=''

print (df6.shape, df7.shape)

prot_conv=df7.to_dict('list')

for i, readline in enumerate(df6['Protein_id']):
    print (i, readline)
    df6.at[i,'Protein_code1']=prot_conv[readline][1]
    df6.at[i,'Protein_code9']=prot_conv[readline][4]

df6.to_csv(fileprotseqcode123_out,index=False,encoding='utf-8')
print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n  duration = ', (time.time()-starttime)/3600, ' hrs \n' )


################# to PaDEL
print('\n',"call PADELdiscriptor")
print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n')

os.chdir(workpath)
toPADEL_file=fileprotseqcode123_out
file1_1d2discriptorOUT=PADELdiscriptor(workpath,toPADEL_file,smiles_label='Smiles',chemical_label='ChEMBL ID')   ### call  PADELdiscriptor

print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())), '\n  duration = ', (time.time()-starttime)/3600, ' hrs \n' )


########################################## DATA binding all Padel-discriptor ##################################
print('\n',"attach PADELdiscriptor 1D 2D to csv", file1_1d2discriptorOUT, ' -> ', fileprotseqcode123_out)
print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())))

os.chdir(workpath)
padel3=pd.read_csv(file1_1d2discriptorOUT,low_memory=False)
padel3_group= padel3.groupby(['Name'])
padel3_group_dict=padel3_group.first().T.to_dict('list')

df7=df6   
df7=df7.reset_index(drop=True)

i=0
df7['padel1d2d']=''
chem_redo=['']
for i, chemid in enumerate (df7['ChEMBL ID']):
    chemid=str(chemid)
    AgonistAnagonistP1N1=df7.at[i,AgonistAnagonist_label]
    try:
        df7.at[i,'padel1d2d']=str(padel3_group_dict[chemid])
    except:
        print ('xxxxxxxxxxx cannot find padel1d2d ',i, chemid,' xxxxxxxxxxx')
        chem_redo.append(str(chemid))
        df7.at[i,AgonistAnagonist_label]=10+AgonistAnagonistP1N1



os.chdir(workpath)
df7.to_csv(fileprotseqcode123_1d2dpadel_out,index=False,encoding='utf-8')

print ('Please check padel1d2d for : ' , chem_redo)

print('\n',time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())),'\n','Total = ', (time.time()-starttime)/60 , ' mins \n\n\n ')




################################### to generate clean data to column ##########################################
timestamp=time.strftime("%Y%m%d%H%M",time.localtime(time.time()))


print(time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())))

PROTEIN_code='Protein_code1'
PROTEIN_code_drop=['Protein_code2','Protein_code3','Protein_code9']
ec50_label='ActivityValue_log_EC50'  
AgonistAnagonist_label='AgonistP1AnagonistN1_label' 



filein=fileprotseqcode123_1d2dpadel_out
#filein='GPCR_AgonistAntagonist_protein_align_ptcode123_padel1d2d20220921_rdkECFP1019.csv'
df8=pd.read_csv(filein, low_memory=False)


df8=df8.loc[(df8[AgonistAnagonist_label]==1) | (df8[AgonistAnagonist_label]==-1) ]


outputpath=homepath+'/Human_AgonistAntagonist/GPCR_separatePROTEIN'


os.chdir(outputpath)

ppp='allprotein'
df_individual, ec50_label=code_to_column(df_temp_cc=df8, PROTEIN_code=PROTEIN_code, PROTEIN_code_drop=PROTEIN_code_drop, proteinname=ppp,ec50_label=ec50_label,AgonistAnagonist_label=AgonistAnagonist_label) 
fileout0='GPCR_to_trainAgonistAntagonist_all.csv'

os.chdir(outputpath)
if len(df_individual)<1 :
    print ('PASS the protein due to no result', ppp) 
else:
    print ('write to file ', ppp , len(df_individual))
    df_individual.to_csv(fileout0,index=False,encoding='utf-8')

print('\n',time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())),'\n','Total = ', (time.time()-starttime)/60 , ' mins \n\n\n ')


##########################################

os.chdir(workpath)
df_totrain=pd.read_csv('GPCR_to_trainAgonistAntagonist_all.csv',low_memory=False)

os.chdir(outputpath)
file=open('protein_list.txt','a')

print ('output to :',outputpath, '\n  Agonist  ', len(df_totrain.loc[(df_totrain[AgonistAnagonist_label]==1)]),'  Antagonist  ', len(df_totrain.loc[(df_totrain[AgonistAnagonist_label]==-1)]),'total: ',len(df_totrain) ,'\n')
df_totrain_group= df_totrain.groupby(['Protein_name'])
i=0
ppp=''
for i, ppp in enumerate (df_totrain_group.first().index):
    ppname=re.sub('\s+','',str(ppp))
    fileout0='GPCR_to_trainAgonistAntagonist_'+ppname+'.csv'
    fileoutP1='GPCR_to_trainAgonist_'+ppname+'.csv'
    fileoutN1='GPCR_to_trainAntagonist_'+ppname+'.csv'
    df_individual0=df_totrain_group.get_group(ppp).reset_index(drop=True)

    df_individualP1=df_individual0.loc[df_individual0[AgonistAnagonist_label]==1]
    df_individualN1=df_individual0.loc[df_individual0[AgonistAnagonist_label]==-1]
    
    os.chdir(outputpath)
    if len(df_individual0)>0:
        df_individual0.to_csv(fileout0,index=False,encoding='utf-8')
    if len(df_individualP1)>0:
        df_individualP1.to_csv(fileoutP1,index=False,encoding='utf-8')
    if len(df_individualN1)>0:
        df_individualN1.to_csv(fileoutN1,index=False,encoding='utf-8')
    print (' write to file ', ppp , len(df_individual0),' Agonist: ',len(df_individualP1),' Antagonist: ',len(df_individualN1))
    ppinfo=ppname+' '+str(len(df_individual0))+' Agonist: '+str(len(df_individualP1))+' Antagonist: '+str(len(df_individualN1))
    file.write(ppinfo+' \n')
    
file.close()
print('\n',time.strftime("%Y/%m/%d %H:%M:%S",time.localtime(time.time())),'\n','Total = ', (time.time()-starttime)/60 , ' mins \n\n\n ')




##### original database:

In [3]:
print("Load original database")
os.chdir(homepath+'/ori_db')
df1=pd.DataFrame()
for f_i in range (0,11):
    filename='gpcrdb_2021_'+str(f_i)+'.csv'
    dfcut=pd.read_csv(filename)
    df1=pd.concat([df1,dfcut],axis=0)
    df1=df1.reset_index(drop=True)
df1

Load original database


/tmp/ipykernel_939001/2921578036.py:6: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  dfcut=pd.read_csv(filename)


,IND,Select all,ChEMBL ID,Receptor,Species,Purchasable,p-value(-log),ActivityType,ActivityRelation,ActivityValue,Unit,Assay Type,Assay Description,Molweight,RotBonds,H don,H acc,LogP,Smiles
0,0,NaN,CHEMBL3403619,s1pr1_rat,Rat,Yes,10.4,EC50,=,0.0,NaN,Funct,Agonist activity at rat S1P1 receptor assesse...,473.0,6,3,7,3.1,C1CCC(C1)C2=C(C=C(C=C2)COC3=CC4=C(C=C3)NC5=C4C...
1,1,NaN,CHEMBL3403619,s1pr1_rat,Rat,Yes,10.4,EC50,=,0.0,NaN,Funct,Agonist activity at rat S1P1 receptor assesse...,473.0,6,3,7,3.1,C1CCC(C1)C2=C(C=C(C=C2)COC3=CC4=C(C=C3)NC5=C4C...
2,2,NaN,CHEMBL3359523,s1pr1_rat,Rat,No,10.3,EC50,=,0.1,NaN,Funct,Agonist activity at rat S1P1 receptor assesse...,482.0,7,1,7,5.4,CC(C)OC1=C(C=C(C=C1)COC2=CC3=C(C=C2)N4CCC(C4=C...
3,3,NaN,CHEMBL3359523,s1pr1_rat,Rat,No,10.3,EC50,=,0.1,NaN,Funct,Agonist activity at rat S1P1 receptor assesse...,482.0,7,1,7,5.4,CC(C)OC1=C(C=C(C=C1)COC2=CC3=C(C=C2)N4CCC(C4=C...
4,4,NaN,CHEMBL3359522,s1pr1_rat,Rat,No,10.1,EC50,=,0.1,NaN,Funct,Agonist activity at rat S1P1 receptor assesse...,439.0,7,1,5,4.2,CC(C)OC1=C(C=C(C=C1)COC2=CC3=C(C=C2)N4CCC(C4=C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
471350,471350,NaN,CHEMBL2069575,gp142_human,Human,No,6.0,EC50,=,970.0,NaN,Funct,Agonist activity at human GPR142 expressed in...,415.0,8,2,5,3.8,C1=CC=C(C=C1)CC(C(=O)NC2=CC=CC(=C2)C3=CC=NC=C3...
471351,471351,NaN,CHEMBL2069575,gp142_human,Human,No,6.0,EC50,=,970.0,NaN,Funct,Agonist activity at human GPR142 expressed in...,415.0,8,2,5,3.8,C1=CC=C(C=C1)CC(C(=O)NC2=CC=CC(=C2)C3=CC=NC=C3...
471352,471352,NaN,CHEMBL2069578,gp142_human,Human,No,6.0,EC50,=,980.0,NaN,Funct,Agonist activity at human GPR142 expressed in...,445.0,9,2,6,3.8,COC1=C(C=C(C=C1)NC(=O)C(CC2=CC=CC=C2)NCC3=CSC=...
471353,471353,NaN,CHEMBL2069578,gp142_human,Human,No,6.0,EC50,=,980.0,NaN,Funct,Agonist activity at human GPR142 expressed in...,445.0,9,2,6,3.8,COC1=C(C=C(C=C1)NC(=O)C(CC2=CC=CC=C2)NCC3=CSC=...
